In [71]:
import tensorflow as tf
import torch
from torch import nn
import numpy as np

In [12]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.20f}".format(x)})
torch.set_printoptions(precision=20)
# x = tf.constant([2.5], dtype=tf.bfloat16)
x = tf.random.uniform([3,4], dtype=tf.float16)
x1 = torch.tensor(tf.cast(x, tf.float32).numpy())
x1 = x1.to(dtype=torch.float16)
print(x)
print(x1)
print(x == x1)

tf.Tensor(
[[0.64257812500000000000 0.24609375000000000000 0.81933593750000000000
  0.70703125000000000000]
 [0.41406250000000000000 0.08886718750000000000 0.96582031250000000000
  0.06054687500000000000]
 [0.32812500000000000000 0.42480468750000000000 0.78125000000000000000
  0.43359375000000000000]], shape=(3, 4), dtype=float16)
tensor([[0.64257812500000000000, 0.24609375000000000000, 0.81933593750000000000,
         0.70703125000000000000],
        [0.41406250000000000000, 0.08886718750000000000, 0.96582031250000000000,
         0.06054687500000000000],
        [0.32812500000000000000, 0.42480468750000000000, 0.78125000000000000000,
         0.43359375000000000000]], dtype=torch.float16)
tf.Tensor(
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]], shape=(3, 4), dtype=bool)


In [4]:
tf.print(x)

[[0.359375 0.1484375 0.9296875 0.9765625]
 [0.609375 0.984375 0.5546875 0.0859375]
 [0.2734375 0.59375 0.84375 0.859375]]


In [75]:
class LayerNorm(tf.Module):
    def __init__(self, weights, biases, eps=1e-12, name=None):
        super().__init__(name)
        # w's and b's are both 1d (768)
        self.weights = weights
        self.biases = biases
        self.eps = eps
    # @tf.function
    def __call__(self, x):
        mean = tf.reduce_mean(x, axis=-1, keepdims=True)
        variance = tf.reduce_mean(tf.square(x - mean), axis=-1, keepdims=True)
        x = (x - mean) / tf.sqrt(variance + self.eps)
        x = self.weights * x + self.biases
        return x


# class PT_LayerNorm(torch.nn.Module):
#     def __init__(self, weights, biases, eps):
#         super().__init__()
#         self.weights = weights
#         self.biases = biases
#         self.eps = eps
#     def forward(self, input):
#         mean = torch.mean(x,)

class PT_LayerNorm(nn.Module):
    def __init__(self, weights, biases, eps):
        super(PT_LayerNorm, self).__init__()
        self.weights = nn.Parameter(weights)
        self.biases = nn.Parameter(biases)
        self.eps = eps

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        variance = (x - mean).pow(2).mean(dim=-1, keepdim=True)
        x = (x - mean) / torch.sqrt(variance + self.eps)
        x = self.weights * x + self.biases
        return x

    
class Dense_v2(tf.Module):
    def __init__(self, in_features:int, out_features:int, weights:tf.Tensor, bias:tf.Tensor=None, name=None):
        super().__init__(name=name)
        self.in_features = in_features
        self.out_features = out_features
        self.w = tf.reshape(weights, (in_features, out_features))
        if bias is None:
            self.b = tf.zeros([1, out_features])
        else:
            self.b = tf.reshape(bias, (1, out_features))
    @tf.function
    def __call__(self, x:tf.Tensor):
        out = tf.matmul(x, self.w)
        return out + self.b

In [2]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.20f}".format(x)})
torch.set_printoptions(precision=20)

In [126]:
# testing layer norm

torch.set_default_device("cuda")
torch.autocast("cpu", enabled=False)
torch.autocast("cuda", enabled=False)

pt_dtype = torch.float16 # test with float maybe
eps = 1e-5
pt_val = {
    'weight':   torch.randn([4], dtype=pt_dtype),
    'bias':     torch.randn([4], dtype=pt_dtype),
    'input':    torch.randn([4], dtype=pt_dtype)
}

tf_val = {}
for key in pt_val:
    tf_val[key] = tf.constant(pt_val[key].cpu().numpy())
    x = tf_val[key] == pt_val[key].cpu()
    assert(tf.reduce_all(x).numpy() == True)

pt_ln = torch.nn.LayerNorm(4, eps=eps)
pt_ln.weight = torch.nn.Parameter(pt_val['weight'])
pt_ln.bias = torch.nn.Parameter(pt_val['bias'])

my_ln = LayerNorm(tf_val['weight'], tf_val['bias'], eps=eps)
my_pt_ln = PT_LayerNorm(pt_val['weight'], pt_val['bias'], eps=eps)

print(pt_ln(pt_val['input']).cpu().detach().numpy())
print(my_pt_ln(pt_val['input']).cpu().detach().numpy())
print(my_ln(tf_val['input']).numpy())

[0.13732910156250000000 0.42211914062500000000 2.33789062500000000000
 -1.24511718750000000000]
[0.13745117187500000000 0.42211914062500000000 2.33789062500000000000
 -1.24609375000000000000]
[0.13745117187500000000 0.42211914062500000000 2.33789062500000000000
 -1.24609375000000000000]


In [160]:
torch_layer = torch.nn.LayerNorm(4, eps=eps) # if the 
torch_layer.weight = torch.nn.Parameter(og_weights)
torch_layer.bias = torch.nn.Parameter(og_bias)

my_layer = LayerNorm(tf_weights, tf_bias, eps=eps)

# tf.keras.backend.set_floatx('float64')
# gamma = tf.constant_initializer(tf_weights.numpy())
# beta = tf.constant_initializer(tf_bias.numpy())
# keras_layer = tf.keras.layers.LayerNormalization(axis=-1, epsilon=eps, gamma_initializer=gamma, beta_initializer=beta)
# print(keras_layer(tf_input))


print(torch_layer(og_input))
# print(torch.nn.functional.layer_norm(og_input, [4], og_weights, og_bias, eps))
print(my_layer(tf_input))



# dont forget to run as keras
# the torch LN might be internally casting as float32

tensor([-0.66259765625000000000,  1.55371093750000000000,
        -1.50390625000000000000,  0.98095703125000000000], device='cuda:0',
       dtype=torch.float16, grad_fn=<NativeLayerNormBackward0>)
tf.Tensor(
[-0.66406250000000000000 1.55468750000000000000 -1.50390625000000000000
 0.98046875000000000000], shape=(4,), dtype=float16)


In [29]:
# testing linear layers
# float16 is off, but float32 and float64 is good
torch.set_default_device("cuda")
pt_dtype=torch.float16

pt_val = {
    'weight':   torch.randn([4,4], dtype=pt_dtype),
    'bias':     torch.randn([4], dtype=pt_dtype),
    'input':    torch.randn([1, 4], dtype=pt_dtype)
}

tf_val = {}
for key in pt_val:
    tf_val[key] = tf.constant(pt_val[key].cpu().numpy())
    x = tf_val[key] == pt_val[key].cpu()
    assert(tf.reduce_all(x).numpy() == True)



pt_linear = torch.nn.Linear(4, 4)
pt_linear.weight = torch.nn.Parameter(pt_val['weight'])
pt_linear.bias = torch.nn.Parameter(pt_val['bias'])

my_linear = Dense_v2(4, 4, tf.transpose(tf_val['weight']), tf_val['bias'])

print(pt_linear(pt_val['input']))
print(my_linear(tf_val['input']))
print("\n", tf.reduce_all(pt_linear(pt_val['input']).cpu().detach().numpy() == my_linear(tf_val['input'])))

tensor([[-1.51855468750000000000, -2.39648437500000000000,
         -0.05068969726562500000,  0.94482421875000000000]], device='cuda:0',
       dtype=torch.float16, grad_fn=<AddmmBackward0>)
tf.Tensor(
[[-1.51757812500000000000 -2.39648437500000000000 -0.05102539062500000000
  0.94482421875000000000]], shape=(1, 4), dtype=float16)

 tf.Tensor(False, shape=(), dtype=bool)
